In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds  # To load the CelebA dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import time


# Define the data directory for CelebA
data_dir = '/path/to/manual/dir'  # Replace with your desired data directory
# Construct the expected file path for CelebA within the data directory
celeb_a_dir = os.path.join(data_dir, 'celeb_a')

# Download the CelebA dataset using TensorFlow Datasets, specifying the manual directory
# and skipping checksum validation initially.
config = tfds.download.DownloadConfig(
    manual_dir=celeb_a_dir,
    register_checksums=False # Temporarily disable checksum validation to allow manual download
)

In [5]:

# Set constants for training
BUFFER_SIZE = 10000       # Buffer size for shuffling the dataset
BATCH_SIZE = 128          # Number of samples per batch
NOISE_DIM = 100           # Dimensionality of the noise vector (latent space)
EPOCHS = 50               # Number of training epochs
IMAGE_SIZE = 64           # Resize images to 64x64
NUM_CHANNELS = 3          # Number of image channels (3 for RGB)

# Directory to save generated images
OUTPUT_DIR = 'gan_celeba_outputs'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Preprocess the images: resize and normalize to [-1, 1]
def preprocess(image):
    # Resize image to IMAGE_SIZE x IMAGE_SIZE
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    # Normalize pixel values to [-1, 1]
    image = (image - 127.5) / 127.5
    return image

# Prepare the dataset for training
def prepare_dataset():
    # Apply preprocessing to each image
    dataset = dataset.map(lambda x: preprocess(x['image']), num_parallel_calls=tf.data.AUTOTUNE)
    # Shuffle and batch dataset
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    # Prefetch for performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Build the Generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*512, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((8, 8, 512)))

    # Upsample layers
    model.add(layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(NUM_CHANNELS, kernel_size=4, strides=1, padding='same', use_bias=False, activation='tanh'))
    return model

# Build the Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=[IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Define loss function and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

# Checkpointing to save model weights
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Generate and save images during training
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(os.path.join(OUTPUT_DIR, f'image_at_epoch_{epoch:04d}.png'))
    plt.close()

# Seed for generating images
seed = tf.random.normal([16, NOISE_DIM])

# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training loop
def train(dataset, epochs):
    for epoch in range(1, epochs + 1):
        start = time.time()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        generate_and_save_images(generator, epoch, seed)

        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print(f'Epoch {epoch}/{epochs} | Gen Loss: {gen_loss:.4f} | Disc Loss: {disc_loss:.4f} | Time: {time.time()-start:.2f} sec')

    generate_and_save_images(generator, epochs, seed)

# Prepare and train the dataset
train_dataset = prepare_dataset()
train(train_dataset, EPOCHS)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


UnboundLocalError: local variable 'dataset' referenced before assignment

In [8]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds  # To load the CelebA dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import time


# Define the data directory for CelebA
data_dir = '/content/'  # Replace with your desired data directory
# Construct the expected file path for CelebA within the data directory
celeb_a_dir = os.path.join(data_dir, 'celeb_a')

# Download the CelebA dataset using TensorFlow Datasets, specifying the manual directory
# and skipping checksum validation initially.
config = tfds.download.DownloadConfig(
    manual_dir=celeb_a_dir,
    register_checksums=False # Temporarily disable checksum validation to allow manual download
)

# Set constants for training
BUFFER_SIZE = 10000       # Buffer size for shuffling the dataset
BATCH_SIZE = 128          # Number of samples per batch
NOISE_DIM = 100           # Dimensionality of the noise vector (latent space)
EPOCHS = 50               # Number of training epochs
IMAGE_SIZE = 64           # Resize images to 64x64
NUM_CHANNELS = 3          # Number of image channels (3 for RGB)

# Directory to save generated images
OUTPUT_DIR = 'gan_celeba_outputs'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Preprocess the images: resize and normalize to [-1, 1]
def preprocess(image):
    # Resize image to IMAGE_SIZE x IMAGE_SIZE
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    # Normalize pixel values to [-1, 1]
    image = (image - 127.5) / 127.5
    return image

# Prepare the dataset for training
def prepare_dataset():
    # Load the CelebA dataset, enabling download
    # and passing the download configuration.
    dataset = tfds.load('celeb_a', split='train', download=True, data_dir=data_dir, download_and_prepare_kwargs={'download_config': config})
    # Apply preprocessing to each image
    dataset = dataset.map(lambda x: preprocess(x['image']), num_parallel_calls=tf.data.AUTOTUNE)
    # Shuffle and batch dataset
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    # Prefetch for performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


# Build the Generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*512, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((8, 8, 512)))

    # Upsample layers
    model.add(layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(NUM_CHANNELS, kernel_size=4, strides=1, padding='same', use_bias=False, activation='tanh'))
    return model

# Build the Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=[IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Define loss function and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

# Checkpointing to save model weights
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Generate and save images during training
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(os.path.join(OUTPUT_DIR, f'image_at_epoch_{epoch:04d}.png'))
    plt.close()

# Seed for generating images
seed = tf.random.normal([16, NOISE_DIM])

# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training loop
def train(dataset, epochs):
    for epoch in range(1, epochs + 1):
        start = time.time()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        generate_and_save_images(generator, epoch, seed)

        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print(f'Epoch {epoch}/{epochs} | Gen Loss: {gen_loss:.4f} | Disc Loss: {disc_loss:.4f} | Time: {time.time()-start:.2f} sec')

    generate_and_save_images(generator, epochs, seed)

# Prepare and train the dataset
train_dataset = prepare_dataset()
train(train_dataset, EPOCHS)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

NonMatchingChecksumError: Artifact https://drive.google.com/uc?export=download&id=1_ee_0u7vcNLOfNLegJRHmolfH5ICW-XS, downloaded to /content/downloads/celeb_a/ucexport_download_id_1_ee_0u7vcNLOfNLegJRHm-XS-m5PzEfg9sSrNxcG3ljG0A7D5ulnonRfaRfoicHDKdU.tmp.843f2b65bf6e41c697a1e7763ed91203/download, has wrong checksum:
* Expected: UrlInfo(size=3.27 MiB, checksum='c6143857c3e2630ac2da9f782e9c1232e5e59be993a9d44e8a7916c78a6158c0', filename='identity_CelebA.txt')
* Got: UrlInfo(size=1.96 KiB, checksum='2ab191514c6fd90ca2e03a2f3f3c867b518cf654925c7cd0072fc42c2f5a184a', filename='download')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror

In [11]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os
import time

# Set data directory
data_dir = '/content/celeb_a'  # Replace with your desired data directory
celeb_a_dir = os.path.join(data_dir, 'celeb_a')

# Define download configuration
config = tfds.download.DownloadConfig(
    manual_dir=celeb_a_dir,
    register_checksums=False  # Allows bypassing checksum validation
)

# Set constants for training
BUFFER_SIZE = 10000
BATCH_SIZE = 64  # Reduced to avoid memory errors; adjust if needed
NOISE_DIM = 100
EPOCHS = 50
IMAGE_SIZE = 64
NUM_CHANNELS = 3

# Directory to save generated images
OUTPUT_DIR = 'gan_celeba_outputs'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Function to preprocess images
def preprocess(image):
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image = (image - 127.5) / 127.5
    return image

# Function to prepare dataset
def prepare_dataset():
    try:
        # Load the CelebA dataset
        dataset = tfds.load('celeb_a', split='train', data_dir=data_dir, download_and_prepare_kwargs={'download_config': config})
        dataset = dataset.map(lambda x: preprocess(x['image']), num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        return dataset
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Error loading dataset: Artifact https://drive.google.com/uc?export=download&id=1_ee_0u7vcNLOfNLegJRHmolfH5ICW-XS, downloaded to /content/celeb_a/downloads/celeb_a/ucexport_download_id_1_ee_0u7vcNLOfNLegJRHm-XS-m5PzEfg9sSrNxcG3ljG0A7D5ulnonRfaRfoicHDKdU.tmp.0bfa0100cd5f415e9b7307125732a714/download, has wrong checksum:
* Expected: UrlInfo(size=3.27 MiB, checksum='c6143857c3e2630ac2da9f782e9c1232e5e59be993a9d44e8a7916c78a6158c0', filename='identity_CelebA.txt')
* Got: UrlInfo(size=1.96 KiB, checksum='9e1b052ca833428d82c45557769bdfae5d899d4844243398974c47fc700c0300', filename='download')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror
Dataset not loaded. Please check the dataset path and format.


In [12]:

# Generator model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8*8*512, use_bias=False, input_shape=(NOISE_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 512)),

        layers.Conv2DTranspose(256, 4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(128, 4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, 4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(NUM_CHANNELS, 4, strides=1, padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, 4, strides=2, padding='same', input_shape=[IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS]),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Conv2D(128, 4, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Conv2D(256, 4, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Conv2D(512, 4, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Define loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

# Checkpointing
checkpoint_dir = './training_checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Generate and save images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(os.path.join(OUTPUT_DIR, f'image_at_epoch_{epoch:04d}.png'))
    plt.close()

# Seed for generating images
seed = tf.random.normal([16, NOISE_DIM])

# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training loop
def train(dataset, epochs):
    if dataset is None:
        print("Dataset not loaded. Please check the dataset path and format.")
        return

    for epoch in range(1, epochs + 1):
        start = time.time()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        generate_and_save_images(generator, epoch, seed)

        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print(f'Epoch {epoch}/{epochs} | Gen Loss: {gen_loss:.4f} | Disc Loss: {disc_loss:.4f} | Time: {time.time()-start:.2f} sec')

    generate_and_save_images(generator, epochs, seed)

# Prepare and train the dataset
train_dataset = prepare_dataset()
train(train_dataset, EPOCHS)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Error loading dataset: Artifact https://drive.google.com/uc?export=download&id=1_ee_0u7vcNLOfNLegJRHmolfH5ICW-XS, downloaded to /content/celeb_a/downloads/celeb_a/ucexport_download_id_1_ee_0u7vcNLOfNLegJRHm-XS-m5PzEfg9sSrNxcG3ljG0A7D5ulnonRfaRfoicHDKdU.tmp.adf10255db1246b09bf62189d072d4c4/download, has wrong checksum:
* Expected: UrlInfo(size=3.27 MiB, checksum='c6143857c3e2630ac2da9f782e9c1232e5e59be993a9d44e8a7916c78a6158c0', filename='identity_CelebA.txt')
* Got: UrlInfo(size=1.96 KiB, checksum='fb0eb9a7bfff0d7ae41116a56f1f4e9e4d8314da4fa8518ca05591083a34da16', filename='download')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror
Dataset not loaded. Please check the dataset path and format.


In [13]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time

# Set constants for training
BUFFER_SIZE = 60000       # Buffer size for shuffling the dataset
BATCH_SIZE = 128          # Number of samples per batch
NOISE_DIM = 100           # Dimensionality of the noise vector (latent space)
EPOCHS = 50               # Number of training epochs
IMAGE_SIZE = 32           # CIFAR-10 images are 32x32
NUM_CHANNELS = 3          # Number of image channels (3 for RGB)

# Directory to save generated images
OUTPUT_DIR = 'gan_cifar10_outputs'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Preprocess the images: normalize to [-1, 1]
def preprocess(image):
    # Normalize pixel values to [-1, 1]
    image = (image - 127.5) / 127.5
    return image

# Load and preprocess CIFAR-10 dataset
def prepare_dataset():
    (train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()
    train_images = train_images.astype('float32')
    train_images = preprocess(train_images)
    dataset = tf.data.Dataset.from_tensor_slices(train_images)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return dataset

# Build the Generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((8, 8, 256)))

    # Upsample layers
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(NUM_CHANNELS, kernel_size=4, strides=1, padding='same', use_bias=False, activation='tanh'))
    return model

# Build the Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=[IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Define loss function and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

# Checkpointing to save model weights
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Generate and save images during training
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0  # Rescale to [0, 1] for display

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(os.path.join(OUTPUT_DIR, f'image_at_epoch_{epoch:04d}.png'))
    plt.close()

# Seed for generating images
seed = tf.random.normal([16, NOISE_DIM])

# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training loop
def train(dataset, epochs):
    for epoch in range(1, epochs + 1):
        start = time.time()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        generate_and_save_images(generator, epoch, seed)

        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print(f'Epoch {epoch}/{epochs} | Gen Loss: {gen_loss:.4f} | Disc Loss: {disc_loss:.4f} | Time: {time.time()-start:.2f} sec')

    generate_and_save_images(generator, epochs, seed)

# Prepare and train the dataset
train_dataset = prepare_dataset()
train(train_dataset, EPOCHS)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step
Epoch 1/50 | Gen Loss: 17.4130 | Disc Loss: 0.0044 | Time: 31.17 sec
Epoch 2/50 | Gen Loss: 0.9769 | Disc Loss: 1.0972 | Time: 19.11 sec
Epoch 3/50 | Gen Loss: 0.9745 | Disc Loss: 1.0784 | Time: 19.30 sec
Epoch 4/50 | Gen Loss: 0.8073 | Disc Loss: 1.4066 | Time: 19.78 sec
Epoch 5/50 | Gen Loss: 0.7962 | Disc Loss: 1.3408 | Time: 19.49 sec
Epoch 6/50 | Gen Loss: 0.6607 | Disc Loss: 1.3511 | Time: 19.55 sec
Epoch 7/50 | Gen Loss: 0.6709 | Disc Loss: 1.4419 | Time: 19.40 sec
Epoch 8/50 | Gen Loss: 0.6252 | Disc Loss: 1.3567 | Time: 19.42 sec
Epoch 9/50 | Gen Loss: 0.6065 | Disc Loss: 1.2755 | Time: 19.45 sec
Epoch 10/50 | Gen Loss: 0.8384 | Disc Loss: 1.2874 | Time: 19.91 sec
Epoch 11/50 | Gen Loss: 0.7687 | Disc Loss: 1.2966 | Time: 19.57 sec
Epoch 12/50 | Gen Loss: 0.7186 | Disc Loss: 1.3123 | Time: 19.47 sec
Epoch 13/50 | Gen Loss: 0.8105 | Disc Loss: 1.2899 | Time: 19.41 sec
Epoch 14/50 | Gen Loss: 0.8650 | Disc Loss: 1.3160 | Time